# CLRNET_DF+NT-(FS,F2F,DFD,DFDC)_train
### Import Python Packages


In [1]:
from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model
import numpy as np
from CLRNet_convlstm import CLRNet
from Utility_functions import create_sequence, FreezeBatchNormalization,AdditionalValidationSets
from datetime import datetime as dt
from DFVDSequence import DFVDSequence
import matplotlib.pyplot as plt
print("yolo")
from sklearn.metrics import classification_report
import pandas as pd
# import shutil
from tensorflow.python.keras import layers
# from src.xception_convlstm import Xception
# print(cv2.__version__)
# import random
# random.seed(32)

dataset_dir='DeepFakeDatasetReal'
# from src.cl_basic import cl_basic
import ipykernel
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

## Load Transfer Dataset

In [2]:
train_video_per_batch=40
val_video_per_batch=25
test_video_per_batch=25
train_frames_per_video=80 #total frames
val_frames_per_video=5 #total frames
test_frames_per_video=5 #total frames
frames_per_video_per_batch=5 #frames in one batch
image_size=128
channel=3
train_augumentation=True
val_augumentation=False
test_augumentation=False
X_train,y_train, class_weights_train = create_sequence(['datasets/TransferLearning/DeepFake/train/',
                                               'datasets/TransferLearning/NeuralTextures/train/',
                                               'datasets/TransferLearning/FaceSwap/train/',
                                               'datasets/TransferLearning/Face2Face/train/'],
                                                      frames_per_video_per_batch,
                                                       train_frames_per_video)

X_val,y_val,class_weights_val=create_sequence(['datasets/DeepFake/val/',
                                              'datasets/NeuralTextures/val/',
                                              'datasets/FaceSwap/val/',
                                              'datasets/Face2Face/val/'],
                                              frames_per_video_per_batch,
                                                    val_frames_per_video)

train_it=DFVDSequence(X_train,y_train,train_video_per_batch,
                      frames_per_video_per_batch,image_size,
                      train_augumentation,True)
val_it=DFVDSequence(X_val,y_val,val_video_per_batch,
                    frames_per_video_per_batch,image_size,
                    val_augumentation,False)

# Load Model

In [3]:
model=CLRNet(input_shape=(frames_per_video_per_batch,image_size, image_size, channel), classes=2, block='bottleneck', residual_unit='v2',
           repetitions=[3, 4, 6, 3], initial_filters=64, activation='softmax', include_top=False,
           input_tensor=None, dropout=0.25, transition_dilation_rate=(1, 1),
           initial_strides=(2, 2), initial_kernel_size=(7, 7), initial_pooling='max',
           final_pooling=None, top='classification')
model.load_weights('CLR_DF+NT_Ebest.h5')
is_training=False
top_k_layers=206
model,df=FreezeBatchNormalization(is_training,top_k_layers,model)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)

# Validating the Model

In [4]:
# frames_per_video_per_batch=5
# frames_per_video=80
# image_size=128
# test_video_per_batch=50
# data_augmentation=False
# training=False
# X_DeepFake,y_DeepFake,class_weights_DeepFake=create_sequence(['datasets/DeepFake/val/','datasets/NeuralTextures/val/'],frames_per_video_per_batch,frames_per_video)
# DeepFake_it=DFVDSequence(X_DeepFake,y_DeepFake,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation,training)
# predictions = model.predict_generator(DeepFake_it, verbose=1)
# predicted_classes = np.argmax(predictions, axis=1)
# true_classes = DeepFake_it.classes
# report = classification_report(true_classes, predicted_classes, digits=4)
# print(report)

# Unfreezing Few Layers for Training

In [5]:
is_training = True
top_k_layers=206
model,df=FreezeBatchNormalization(is_training,top_k_layers,model)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# adam_fine=Adam()
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)

# Compiling Model

In [6]:
model_checkpoint_callback = ModelCheckpoint(
    filepath="CLR_DF+NT-(FS,F2F,DFD,DFDC)_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

csv_logger = CSVLogger(
    "CLR_DF+NT-(FS,F2F,DFD,DFDC)_train.csv",
    append=True,
    separator=',')

reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=7,
    verbose=1,
    min_delta=0.0001,
    mode='min')


callbacks_list = [csv_logger,model_checkpoint_callback,reduce_lr_loss] 

# _id = dt.now().strftime("%y-%m-%d-%H_%M")
# save_dir=os.path.join(os.getcwd(),'models','transfer_FS-DF-F2F-NT-DFD','CLRNet','DF-F2F-NT-DFD',_id)
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

# filepath=os.path.join(save_dir,"{epoch:02d}-{acc:.2f}"+".hdf5")
# checkpoint = ModelCheckpoint(filepath)#, monitor='acc', verbose=2, save_best_only=True, mode='max')
# # csv_logger = CSVLogger(os.path.join(save_dir,"training.csv"), append=True, separator=',')
# # earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
# # mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
# # reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')


# # earlyStop=EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
# # callbacks_list = [csv_logger]

# csv_logger = AdditionalValidationSets([(val_it_0,'valFS'),(val_it_1,'valDF'),(val_it_2,'valF2F'),(val_it_3,'valNT'),(val_it_4,'valDFD'),(test_it_0,'testFS'),(test_it_1,'testDF'),(test_it_2,'testF2F'),(test_it_3,'testNT'),(test_it_4,'testDFD')],os.path.join(save_dir,"training.csv"))
# # csv_logger = AdditionalValidationSets([(val_it_0,'val0'),(val_it_1,'val1'),(val_it_2,'val2'),(val_it_3,'val3')],os.path.join(save_dir,"training.csv"))

# callbacks_list = [csv_logger,checkpoint]      

# Training

In [7]:
model.fit_generator(generator=train_it,
                    epochs=150,
                    validation_data=val_it,
                    callbacks=callbacks_list,
                    shuffle=False,
                    class_weight=class_weights_train,
                    verbose=1)
model.save_weights("CLR_DF+NT-(FS,F2F,DFD,DFDC)_E150.h5")